<center><img alt="" src="images/Cover_UnMuh_2021.png"/></center> 

# <center><font color="black">Implementasi Deep Learning Bagian ke-01 <br> http://bit.ly/unmuh-A-2021</font></center>
## <center><font color="black">(C) Taufik Sutanto - 2021 <br> tau-data Indonesia ~ https://tau-data.id/</font></center>

## <font color="blue">Outline Module Implementasi A :</font>

* getting ready with Deep Learning & GPU
* Deep learning for Binary Classification
* Deep Learning for Multi-Classification
* Deep Learning for Regression
* Deep Learning for Clustering
* Hyperparameter Optimization & Cross Validation

# Instalasi Lokal GPU Support (TensorFlow-CUDA) untuk model Deep Learning. 

* Link ini akan membantu menyesuaikan versi CUDA dan CudNN yang tepat untuk semua versi TensorFlow.  Hati-hati!!!.... requirement CUDA dan CudNN berbeda antara Linux & Windows (Walau versi tensorflow-nya sama!!!).
* Berikut Versi Keras-TensorFlow yang bersesuaiannya: https://docs.floydhub.com/guides/environments/
Download Cuda dan CudNN yang bersesuaian (seringnya BUKAN versi terakhir) dari sini (need to register):
* https://developer.nvidia.com/cuda-downloads
* https://developer.nvidia.com/rdp/cudnn-archive
 - Copy all files ke instalasi path cuda computing toolkit
 - https://docs.nvidia.com/deeplearning/cudnn/install-guide/index.html 
* Restart

* Setelah install Cuda/Cudnn, jika compiler terinstall dengan baik, maka perintah **pip install --upgrade tensorflow-gpu** bisa digunakan di terminal/command prompt.

[Linux]:  https://www.tensorflow.org/install/source#tested_build_configurations 

[Windows]:  https://www.tensorflow.org/install/source_windows 

Untuk "PyTorch" cenderung lebih mudah: https://pytorch.org/get-started/locally/

# Deep Learning di Google Colaboratory (Recommended*)

* Free with GPU (& TPU) support (Max run ~ 10jam)
* Login dengan Username dan password Google
* Kunjungi https://colab.research.google.com
* New Python 3 Notebook, rename/save Notebook 

## Runtime>Change runtime type and select GPU as Hardware accelerator.

# Ingat di Google Colab/Jupyter Notebook Urutan eksekusi Cell Penting

### Jangan ada yang terlewat.
### Untuk Memudahkan yakinkan "side-by-side" antara Zoom dan Google Colab

In [ ]:
# Importing some supporting libraries
import pickle, numpy as np, warnings; warnings.simplefilter('ignore')
!mkdir data

In [ ]:
import tensorflow as tf

print("TensorFlow version = ", tf.__version__)
if tf.test.is_built_with_cuda():
    physical_devices = tf.config.list_physical_devices('GPU') 
    print("CUDA enabled TF, Num GPUs:", len(physical_devices), physical_devices) 
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
    except:
        pass

# Deep Learning for Sentiment Analysis

<img alt="" src="images/movies.jpg "/>

* Di contoh pertama kita akan menggunakan data IMDB.
* Data ini relatif "mudah" dan cocok untuk pemula.
* Data berupa Text, namun dapat langsung diakses bentuk labeled-nya
* Terdiri dari review **Sentimen** positif dan negatif dari film-film.

In [ ]:
from tensorflow.keras.datasets import imdb

try:
    f = open('data/imdb.pckl', 'rb') # Menghindari downloading data berkali-kali
    (train_data, train_labels), (test_data, test_labels) = pickle.load(f); f.close()
except:
    (train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
    # num_words=10000 artinya 10,000 kata yang sering muncul. 
    f = open('data/imdb.pckl', 'wb') # save ke lokal drive
    pickle.dump(((train_data, train_labels), (test_data, test_labels)), f); f.close()
'Done'

# Tips

### Di Data Science, Machine Learning, & AI selalu perhatikan struktur data input dan output dengan baik.

In [ ]:
type(train_data), train_data.shape

In [ ]:
# Bagaimana memahami struktur data ini?
train_data[:2]

In [ ]:
# Coba duga makna angka-angka ini
type(train_data[0]), len(train_data[0]), len(train_data[1])

In [ ]:
# word_index is a dictionary mapping words to an integer index
word_index = imdb.get_word_index()
# We reverse it, mapping integer indices to words
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# We decode the review; note that our indices were offset by 3
# because 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])
decoded_review

# Struktur Data, Struktur Data, Struktur Data

* ANN/DL via DL tidak dapat menerima input (domain) list dari integer.
* Input data harus diubah terlebih dahulu menjadi **Tensor**
* Lakukan "Pad" agar semua memiliki panjang yang sama.
* Kemudian Rubah ke Tensor
* Gunakan Embedding layer (word2Vec) untuk mendapatkan tensor dari dokumen yang akan dioptimasi untuk klasifikasi.
* Struktur Data input cocok/siap untuk digunakan pada layer/network DL.

### Masih belum jelas? Mari kita bahas perlahan.

# Setiap kata akan masuk ke network kita

* Bayangkan setiap kata sebagai variabel/feature/indicator sebagaimana Vector Space Model

<ul>
	<li>Misal kita kembali menggunakan penjelasan kata per kata yang masuk ke network kita</li>
	<li><img alt="" src="images/lstm_words.png" /></li>
</ul>


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

toy_data = ['Deep Learning sedikit menantang awalnya.','Bagaimana dengan akhirnya? :)',
            'Dengan doa dan usaha Deep Learning sedikit demi sedikit akan dimengerti :)']

tokenizer = Tokenizer()
tokenizer.fit_on_texts(toy_data)
sequences = tokenizer.texts_to_sequences(toy_data)
sequences

In [ ]:
tokenizer.word_index

## pad_sequences di Keras
* Mirip VSM ==> kolom dengan panjang konstan
* Tapi apa bedanya ya?

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X = pad_sequences(sequences)
print(X.shape)
X

# Struktur datanya sudah siap untuk menjadi input untuk layer Word Embedding:

<img alt="" src="images/we.png" />

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding

A = 15 # "Lebih Besar" dari Banyaknya "kata unik" di data training (= 14+1)
B = 2 # Banyak neuron di layar embedding = panjang vektor representasi kata di WE
# For simplicity reason, karena ini hanya "toy data" kita pakai = 2
C = 11 # = X.shape[1] yaitu panjang vector "pad_sequences"
model = Sequential()
model.add(Embedding(A, B, input_length=C))
# Selesai ... pada aplikasinya nanti layer embedding ini akan "disambungkan" dengan
# Layer classifier DL. Namun untuk saat ini mari kita "outputkan" WE-nya seperti
# yang kita lakukan sebelumnya di Gensim
'Done'

In [ ]:
# Cell ini hanya untuk "mengintip" hasil WE-nya
model.compile('rmsprop', 'mse')
WE = model.predict(X)
print(WE.shape)
WE # ==> It's a Tensor! ... Hence TensorFlow :)
# The tensors are flowing throughout the layers and neurons
# like river to the sea :)

# Dengan Word Embeding Kata Tersebut dirubah menjadi angka

* Bayangkan setiap kata menjadi vector
* Sekumpulan kata (misal review) berarti sebuah Matrix.
* Kumpulan Review berarti Tensor

<h1 id="Kata-kata-tersebut-kemudian-dirubah-menjadi-vector-angka">Kata-kata tersebut kemudian dirubah menjadi vector angka</h1>

<ul>
	<li>Ingat saat pembahasan word embedding</li>
	<li><img alt="" src="images/LSTM_kataAngka.gif" style="width: 800px; height: 211px;" /></li>
</ul>


# kembali ke Data Movie Review (IMDB) kita

In [ ]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))# Create an all-zero matrix of shape (len(sequences), dimension)
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # set specific indices of results[i] to 1s
    return results

x_train = vectorize_sequences(train_data) # Our vectorized training data
x_test = vectorize_sequences(test_data) # Our vectorized test data
# Our vectorized labels
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')
'Done'

# Network sederhana kita

* Karena permasalahan kita sederhana kita bisa coba dengan bentuk jaringan sederhana terlebih dahulu: Layer Dense dan fungsi aktivasi ReLu
* Parameter 16 di "Dense" artinya banyak hidden unit.
* variasi parameter yang bisa dibuat: Berapa banyak layer? dan berapa banyak hidden unit?

<img alt="" src="images/3_layer_network.png"/>

# Apa itu Dense?

<img alt="" src="images/Sparse_Dense_Network.png" style="width: 600px; height: 463px;" />

* Matematikawan menyebut ini "Complete Graph" di teori Graph.
* Apa maksudnya "Sequential"? ==> _linear stack of layers_ [Mirip "Lego"]

# Sparse <=> Dropout

<p><img alt="" src="images/dropout.png" style="width: 600px; height: 337px;" /></p>

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
'Done'

# Fungsi Aktivasi

<ul>
	<li>Sekedar merubah kodomain hasil summasi ke rentang nilai tertentu.</li>
	<li><img alt="" src="images/act_func.png" style="width: 760px; height: 206px;" /></li>
</ul>

<p><img alt="" src="images/6_JST_Actv.png" style="width: 756px; height: 625px;" /></p>

<h1 > Contoh Fungsi tanh memetakan [-Inf, Inf] ke [-1, 1]</h1>

<p><img alt="" src="images/tanh_act.gif" style="width: 800px; height: 317px;" /></p>

<h1 >Contoh 2: Fungsi Aktivasi Sigmoid yang bisa digunakan untuk menginat dan melupakan (mantan? :) )</h1>

<ul>
	<li>Fungsi sigmoid mirip dengan tanh, namun intervalnya adalah [0, 1].</li>
	<li>Semakin dekat ke 0 ==&gt; melupakan (forget): informasi tidak relevan.</li>
	<li>Semakin dekat ke 1 ==&gt; mengingat: informasi relevan/penting.</li>
	<li><img alt="" src="images/sigmoid.gif" style="width: 800px; height: 317px;" /></li>
</ul>

# memangnya Tanpa fungsi aktivasi apa yang akan terjadi?

* Tanpa fungsi aktivasi, weights bisa membesar tak berbatas ketika iterasinya berjalan

<p><img alt="" src="images/tanpa_tanh.gif" style="width: 800px; height: 106px;" /></p>

# Dengan Fungsi Aktivasi, nilai weights selalu terbatas (misal) di -1 dan 1, namun tingkat kepentingan weight tetap terjaga.

* Perhatikan nilai weight yang pertama
<img alt="" src="images/dgn_tanh.gif" style="width: 800px; height: 106px;" />

In [ ]:
# Output Layer - Perhatikan pada cell ini Network belum dijalankan, baru di define
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

model.compile(optimizer='adam',
              loss=losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy])
"Done"

# Loss, Optimizer, & metric

<ol>
	<li><strong>Loss function (objective function)</strong> &mdash; fungsi yang akan di minimize. Hasilnya merepresentasikan tingkat sukses pada setiap iterasi.<br />
	<a href="https://keras.io/losses/" target="_blank">https://keras.io/losses/&nbsp;</a></li>
	<li><strong>Optimizer&nbsp;</strong>&mdash; Berfungsi untuk menentukan bagaimana (weights) di network akan di update berdasarkan loss-functionnya. (e.g. variasi dari SGD)<br />
	<a href="https://keras.io/optimizers/" target="_blank">https://keras.io/optimizers/</a></li>
	<li><strong>Metrics&nbsp;</strong>&mdash; Satuan evaluasi<br />
	<a href="https://keras.io/metrics/" target="_blank">https://keras.io/metrics/</a></li>
</ol>



<h2 id="Loss-function-and-Error">Loss function and Error</h2>

<p><img alt="" src="images/low-high-bias-variance.png" style="width: 600px; height: 244px;" /></p>


# Memilih Fungsi Aktivasi dan Fungsi Loss

<img alt="" src="images/memilih_loss_dan_activation.png" style="width: 600px; height: 368px;" />

<p><big><strong>Binary Cross Entropy</strong></big></p>

<p><img alt="" src="images/binary_cross_entropy.png" style="width: 600px; height: 173px;" /></p>

<p><big><strong>Minimum Squared Error</strong></big></p>

<p><img alt="" src="images/mse.png" style="width: 300px; height: 106px;" /></p>

<p><img alt="" src="images/mse_mean.png" style="width: 600px; height: 323px;" /></p>


<table border="1" cellpadding="1" cellspacing="1" style="width: 500px">
	<tbody>
		<tr>
			<td><img alt="" src="images/contours_evaluation_optimizers.gif" /></td>
			<td><img alt="" src="images/saddle_point_evaluation_optimizers.gif" /></td>
		</tr>
	</tbody>
</table>

Optimizer : 
<a href="http://ruder.io/optimizing-gradient-descent/" target="_blank">http://ruder.io/optimizing-gradient-descent/&nbsp;</a>


# Di Deep learning Common Practicenya Data dibagi 3: 
# Training, Validation, dan Test

In [ ]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

# Apa ya Epoch di Deep Learning?

<p><img alt="" src="images/epoch.png" style="width: 600px ; height: 230px" /></p>

## Saatnya Run model DL kita

In [ ]:
# DL baru dijalankan di cell ini
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val, y_val))

# Struktur Data, Struktur Data, Struktur Data

In [ ]:
history_dict = history.history
print(type(history_dict))
history_dict.keys()
# Tapi apa values dari dictionary ini?

In [ ]:
# Sebaiknya di visualisasi
import matplotlib.pyplot as plt

acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')# "bo" is for "blue dot"
plt.plot(epochs, val_loss, 'b', label='Validation loss')# b is for "solid blue line"
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(); plt.show()

In [ ]:
plt.clf()   # clear figure
acc_values = history_dict['binary_accuracy']
val_acc_values = history_dict['val_binary_accuracy']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# bagaimana memaknai graphics diatas?

### Stop di iterasi ke-3

In [ ]:
# Re-Run dengan informasi sebelum ini
# Dijadikan satu biar lebih jelas

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)

In [ ]:
# Akurasi 88.5 ... cukup baik, tapi can be better
# Kita akan coba improve dengan LSTM di Sesi berikutnya.
results

In [ ]:
# Jika kita ingin prediksi ke Data Test
prediksi = model.predict(x_test)
prediksi[:7]
# beberapa prediksi sangat yakin = 0.9999

# Latihan (hingga selesai waktu Sesi ke-02)

* Coba hidden layers 1 dan 3 ... Mana yang terbaik?
* Sebelumnya kita menggunakan 16 unit, coba 8, 32, dan 64. Apa akibatnya?
* Coba tanh ketimbang relu sebagai fungsi aktivasinya. Apakah lebih baik?
* Coba dengan data Lain, misal data dibawah ini.

## Loading Dataset dari Modul
contoh dataset dokumen lain yang cukup tenar: &quot;20 NewsGroup&quot;

<img alt="" src="images/6_20News.jpg" style="height: 300px ; width: 533px" />
<p><a href="http://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html#sklearn.datasets.fetch_20newsgroups" target="_blank">http://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html#sklearn.datasets.fetch_20newsgroups</a></p>

<p><strong>Categories </strong>=&nbsp;</p>
<pre>
[&#39;alt.atheism&#39;, &#39;comp.graphics&#39;, &#39;comp.os.ms-windows.misc&#39;, &#39;comp.sys.ibm.pc.hardware&#39;, &#39;comp.sys.mac.hardware&#39;,
 &#39;comp.windows.x&#39;, &#39;misc.forsale&#39;, &#39;rec.autos&#39;, &#39;rec.motorcycles&#39;, &#39;rec.sport.baseball&#39;, &#39;rec.sport.hockey&#39;,
 &#39;sci.crypt&#39;, &#39;sci.electronics&#39;, &#39;sci.med&#39;, &#39;sci.space&#39;, &#39;soc.religion.christian&#39;, &#39;talk.politics.guns&#39;,
 &#39;talk.politics.mideast&#39;, &#39;talk.politics.misc&#39;, &#39;talk.religion.misc&#39;]</pre>


In [ ]:
# Petunjuk : Gunakan cara merubah text ke Pad-Sequence seperti yang sudah dibahas sebelumnya.
from sklearn.datasets import fetch_20newsgroups

try:
    f = open('data/20newsgroups_bin.pckl', 'rb') # Menghindari downloading data berkali-kali
    Y, D = pickle.load(f); f.close()
except:
    categories = ['talk.politics.misc',  'rec.autos']
    data = fetch_20newsgroups(subset='train', categories=categories,remove=('headers', 'footers', 'quotes'))
    Y = data.target
    D = [doc for doc in data.data]
    f = open('data/20newsgroups_bin.pckl', 'wb') # save ke lokal drive
    pickle.dump((Y, D), f); f.close()
    
'Done'

In [ ]:
# Jawaban disini


<h1>End of Module Implementasi Deep Learning 01</h1>

<hr />
<img alt="" src="images/DL_meme.jpg"/>